In [2]:
from numpy import hstack, ones, array, mat, tile, reshape, squeeze, eye, asmatrix
from numpy.linalg import inv
from pandas import read_csv, Series 
from scipy.linalg import kron,pinv
from scipy.optimize import fmin_bfgs,fmin
import numpy as np
import statsmodels.api as sm
import pandas as pd

In [3]:
def iter_print(params):
    """ callback for print"""
    global iteration, lastValue, functionCount
    iteration += 1
    print('Func value: {0:}, Iteration: {1:}, Function Count: {2:}'.format(lastValue, iteration, functionCount))

In [15]:
def gmm_objective(params, y, x, Winv, out=False):
    """ 计算系数迭代的目标函数 （最小） """
    global lastValue, functionCount
    T,N = y.shape
    T,K = x.shape
    beta = squeeze(array(params))
    beta = reshape(beta,(N,K))
    y_pred = x @ beta.T
    e = y - y_pred # 残差
    e_ = kron(e,ones((1,K))) # 将残差复制K列
    x_ = tile(x,N)  # 将残差复制N列
    moments = e_ * x_ # 计算矩条件  Z*e
    
    avgMoment = moments.mean(axis=0)    
    J =  mat(avgMoment) * mat(Winv) * mat(avgMoment).T
    
    J = J[0,0]
    lastValue = J
    functionCount += 1
    if not out:
        return J
    else:
        return J, moments

In [16]:
def gmm_G(params, y, x,Winv):
    """ 计算 gmm_objective 对系数的导数  """
    T,N = y.shape
    T,K = x.shape
    d = np.zeros((N*K,N*K))
    ffp = (x.T @ x) / T
    d[:(N*K),:(N*K)]=kron(eye(N),ffp)    
    return d

In [8]:
# 读取数据
path= 'auto.csv'
data= pd.read_csv(path)
data['constant']=1 # 增加常数项
data = data[['mpg','gear_ratio','turn','constant']].copy()
x= data[['gear_ratio','turn','constant']].copy().values
y= data[['mpg']].values

In [20]:
# 第一次迭代过程
init_beta= [1,1,1]  # 初始系数
startingVals = np.array(init_beta)  
Winv = np.eye(startingVals.shape[0]) # 初始权重矩阵
args = (y, x, Winv)

iteration = 0
lastValue = 0
functionCount = 0

step1opt = fmin_bfgs(gmm_objective, startingVals, args=args, callback=iter_print)
step1opt


Func value: 35412411.70559802, Iteration: 1, Function Count: 10
Func value: 2553830.3099367493, Iteration: 2, Function Count: 15
Func value: 873048.3384790673, Iteration: 3, Function Count: 25
Func value: 180.33067937863746, Iteration: 4, Function Count: 30
Func value: 0.7332242554405092, Iteration: 5, Function Count: 35
Func value: 0.07773717923544206, Iteration: 6, Function Count: 60
Func value: 0.022250473701542057, Iteration: 7, Function Count: 70
Func value: 0.02148083502644489, Iteration: 8, Function Count: 80
         Current function value: 0.021481
         Iterations: 8
         Function evaluations: 537
         Gradient evaluations: 105


array([ 2.2052607 , -0.82234034, 47.26830392])

In [21]:
# 第一次迭代结果
out = gmm_objective(step1opt, y, x, Winv, out=True) 
T = y.shape[0]
out = out[1]-out[1].mean() # 得到去中心化的矩平均
S = out.T @ out/T
Winv2= np.linalg.inv(S)
args = (y, x, Winv2)

iteration = 0
functionCount = 0

step2opt = fmin_bfgs(gmm_objective, step1opt, args=args, callback=iter_print)
beta = step2opt
beta

Func value: 0.4930986376357178, Iteration: 1, Function Count: 15
Func value: 0.4586593214293435, Iteration: 2, Function Count: 25
Func value: 0.3419069060672252, Iteration: 3, Function Count: 35
Func value: 0.1467328348439741, Iteration: 4, Function Count: 40
Func value: 1.947218670663357e-10, Iteration: 5, Function Count: 45
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 5
         Function evaluations: 45
         Gradient evaluations: 9


array([ 3.03286885, -0.73305195, 41.2181289 ])

In [22]:
##-------- 计算标准误
out = gmm_objective(step2opt, y, x, Winv2, out=True) 
# 计算倒数d
G = gmm_G(step2opt, y, x,Winv2) 
# 矩条件
mom = out[1]
# 计算 矩协方差矩阵
out= out[1]-out[1].mean() 
S_= out.T @ out / T
#  计算系数的协方差矩阵
vcv = pinv(G @ pinv(S_) @ G.T)/T
# 提取对角线的方差
diag_v= np.diag(vcv)
# 计算标准误
np.sqrt(diag_v) 

array([1.50166502, 0.11797213, 8.39674751])